# [Beartooth Least Cost Path Ultra Marathon](https://beartoothleastcostpath.github.io/)

## 1. Determine all possible paths
Using a bit of discrete math, we can envisage the solution must exist in a set of permutations in the given set of Beartooth Mountain Range peaks

In [1]:
from collections import Counter
from itertools import permutations 
from btlcp.geo import waypoints
from numpy import array

# we will count the number of discrete paths to traverse
possibilities = set()

# convert the set to a list because order matters
peaks = list([peak for peak in waypoints()])

# the total number of waypoints that must be visited
num_peaks = len(peaks)

# make a list of indexes because calculating permutations is faster that way
peak_indexes = array([i for i in range(0, num_peaks)])

# use the indexes to find all possible permutations a backpacker can take
path_permutations = permutations(peak_indexes)

for path in path_permutations:
    possibilities.add(path)

path_permutations = None

print(f"There are {len(possibilities)} possible paths to choose from.")

There are 3628800 possible paths to choose from.


Perhaps the most naive attempt would be to find the "least cost" as a function of distance traveled

In [2]:
from numpy import array
from pyproj import Geod

def calc_euclidian_distance(arr = None):
    arr = arr or array()
    
    g = Geod(ellps='WGS84')

    src = arr[0]
    distances = []

    for i in range(1, len(arr)):
        dst = arr[i]
        _, _, dist = g.inv(src[1], src[0], dst[1], dst[0])
        distances.append(dist)
        src = dst
    
    return array(distances)


In [26]:
from pandas import DataFrame
from numpy import array, nan

possible_paths = dict(
    indexes=[],
    # for display
    name=[],
    # for calc
    latlon=[],
    # heuristic
    segment_distance=[],
    total_distance=[],
)

# max_iter = 100

for (index, possibility) in enumerate(possibilities):
    # if index >= max_iter:
    #     break
    by_name = array([peaks[i][0] for i in possibility])
    by_latlon = array([peaks[i][1] for i in possibility])
    by_distance = calc_euclidian_distance([peaks[i][1] for i in possibility])
    by_total_distance = by_distance.sum()
    possible_paths['indexes'].append(possibility)
    possible_paths['name'].append(by_name)
    possible_paths['latlon'].append(by_latlon)
    possible_paths['segment_distance'].append(by_distance)
    possible_paths['total_distance'].append(by_total_distance)


df = DataFrame(possible_paths)

In [65]:

least_cost_by_euclidean_distance = []

shortest_paths_indexes = df[df.total_distance == df.total_distance.min()].indexes

for i, indexes in enumerate(shortest_paths_indexes):
    least_cost_by_euclidean_distance.append([])
    for index in indexes:
        least_cost_by_euclidean_distance[i].append(peaks[index])


print("By euclidean total distance, the least cost path is a tie between the following routes:\n")

msgs = []
for path in least_cost_by_euclidean_distance:
    msg = " -> ".join([p[0] for p in path])
    print(msg)

By euclidean total distance, the least cost path is a tie between the following routes:

Silver Run Peak -> Beartooth Mountain -> Whitetail Peak -> Castle Mountain -> Castle Rock Spire -> Castle Rock Mountain -> Mount Peal -> Tempest Mountain -> Granite Peak -> Mount Wood
Mount Wood -> Granite Peak -> Tempest Mountain -> Mount Peal -> Castle Rock Mountain -> Castle Rock Spire -> Castle Mountain -> Whitetail Peak -> Beartooth Mountain -> Silver Run Peak
